## Data Prep and Loading
0. Download the audo features from the `00_Audio_API_Extract.ipynb`
1. Load playlists from zip
2. Create `playlists_track_string` where tracks are a string to be parsed by tfrecords
3. Join the audio data

In [ ]:
# !gsutil cp gs://matching-engine-content/spotify-million-playlist/spotify_million_playlist_dataset.zip .
# !unzip spotify_million_playlist_dataset.zip

In [ ]:
# import os
# import json

# data_files = os.listdir('data')
# # PROJECT_ID = 'jtotten-project'
# bq_dataset = 'mdp_eda'

# for filename in data_files:
#   with open(f'data/{filename}') as f:
#     print(f)
#     json_dict = json.load(f)
#     df = pd.DataFrame(json_dict['playlists'])
#     df.to_gbq(
#     destination_table=f'{bq_dataset}.playlists', 
#     project_id=f'{PROJECT_ID}', # TODO: param
#     location='us-central1', 
#     progress_bar=True, 
#     reauth=True, 
#     if_exists='append'
#     ) 

In [2]:
%%bigquery
create or replace table jtotten-project.spotify_mpd.playlists_track_string as 
(
SELECT * FROM `jtotten-project.mdp_eda.playlists`)

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1462.28query/s]                        


""


## Need a cross-join with songs in playlist for training tuple

`playlist_train` is the cross - joined - done (need to find code)

next, join tracks to `track_audio`

finally, join the `tracks` string field from above `spotify_mpd.playlists_track_string`

In [ ]:
%%bigquery
create or replace table jtotten-project.spotify_mpd.playlists as (
with json_parsed as (SELECT *, JSON_EXTRACT_ARRAY(tracks) as json_data FROM `jtotten-project.mdp_eda.playlists` )

select json_parsed.* except(tracks, duration_ms, json_data),
JSON_EXTRACT(jsn_line, "$.pos") as pos, 
JSON_EXTRACT(jsn_line, "$.artist_name") as artist_name,
JSON_EXTRACT(jsn_line, "$.track_uri") as track_uri,
JSON_EXTRACT(jsn_line, "$.artist_uri") as artist_uri,
JSON_EXTRACT(jsn_line, "$.track_name") as track_name,
JSON_EXTRACT(jsn_line, "$.album_uri") as album_uri,
JSON_EXTRACT(jsn_line, "$.duration_ms") as duration_ms,
JSON_EXTRACT(jsn_line, "$.album_name") as album_name,

from json_parsed CROSS JOIN UNNEST(json_parsed.json_data) as jsn_line
)

### Join Audio Data (see Audio Api Extraction)

In [7]:
%%bigquery
create or replace table `jtotten-project.spotify_mpd.audio_stage` as (
    SELECT 
    a.pid,
    b.* 
FROM `jtotten-project.spotify_mpd.playlists` a
inner join `jtotten-project.spotify_mpd.track_audio` b
on a.track_uri = b.uri
)

Query complete after 0.00s: 100%|██████████| 4/4 [00:00<00:00, 4059.33query/s]                        


""


### Incorporate playlist string data along with playlist metadata

In [8]:
%%bigquery
create or replace table `jtotten-project.spotify_mpd.spotify_2tdata` as (
    SELECT
    a.*,
    c.tracks, 
    c.name as pl_name, 
    c.collaborative, 
    c.modified_at as modified_at_playlist, 
    c.num_tracks, 
    c.num_albums, 
    c.num_followers, 
    c.num_edits, 
    c.num_artists, 
    c.description
FROM `jtotten-project.spotify_mpd.audio_stage` a
inner join `jtotten-project.spotify_mpd.playlists_track_string` c
on a.pid = c.pid
)

Query complete after 0.00s: 100%|██████████| 14/14 [00:00<00:00, 10938.94query/s]                      


""
